# VDL Exericse 2 (Bonus Task 2.6)

**Group Name:** ...


**Participants:**

- Name 1 (Matrikl. Nr. 1)
- Name 2 (Matrikl. Nr. 2)
- ...

__Complete all sub-tasks marked with ## TODO ## and submit the filled notebook on OLAT__ \
__Using a GPU is recommended here__

### Dog Breeds Identification with Transfer Learning ###
Aim of this notebook is to implement the concept of transfer learning to train a bigger dataset. We try to compete on a well-known competiton on Kaggle known as Dog Breeds Identification. Read more about it here:

https://www.kaggle.com/c/dog-breed-identification/overview



To train a model on the Dog breeds dataset using transfer learning and submit your results to Kaggle.
Note: Below notebook gives some tips to run the code in pytorch. 

In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [ ]:
###########################################################################################
## TODO: Register on Kaggle with your respepective group name                            ##
##                                                                                       ##
##                                                                                       ##
##         Group Name:                                                                   ##
##        Kaggle Page:                                                                   ##
##                                                                                       ##
##                                                                                       ##
###########################################################################################
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import matplotlib.pyplot as plt
import pandas as pd
import os
import sys
import shutil

use_cuda = torch.cuda.is_available()

In [ ]:
###########################################################################################
## Mount Google Drive and define paths                                                   ##
###########################################################################################
from google.colab import drive
drive.mount('/content/drive')

# TODO: Define a path inside your Google Drive for this project
project_path = '/content/drive/My Drive/' + ''

# TODO: Change current directory (Hint: use `cd`) to the prject_path


# Define dataset locations
dir_root = "data/"
dir_train = dir_root + "train/"
dir_test = dir_root + "test/"
dir_converted_train = dir_root + "converted/train/"
dir_converted_valid = dir_root + "converted/valid/"

In [ ]:
###########################################################################################
## TODO: Download the Dog-Breeds dataset from the Kaggle competition link mentioned      ##
## above and save it in 'data' subfolder inside the project_path.                        ##
###########################################################################################



In [ ]:
###########################################################################################
## Make your dataset to and dataloaders for the  test data.                              ##
##                                                                                       ##
## HINT: One can (1) make their own custom dataset and dataloaders using the CSV file or ##
## (2) convert the Dog-breed training dataset into Imagenet Format, where all images of  ##
## one class are in a folder named with class as in the below given format. Standard     ##
## Pytorch Datasets and Dataloaders can then be applied over them                        ##
##   Root                                                                                ##
##   |                                                                                   ##
##   |---Class1 ___Img1.png                                                              ##
##   |          ___Img2.png                                                              ##
##   |                                                                                   ##
##   |---Class2 ___Img3.png                                                              ##
##   |          ___Img4.png                                                              ##
##   |....                                                                               ##
##   |....                                                                               ##
###########################################################################################
import torchvision
import torchvision.transforms as transforms
import config as cf
import transform as tf

from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import ImageFolder


###########################################################################################
## TODO: Define the dataloaders and load train and test sets                             ##
###########################################################################################


###########################################################################################
## TODO: Split train data into train and val sets and make dataloaders.                  ##
###########################################################################################



# Print length of all three datasets, and shape of one sample for verification

__Train famous Alexnet model on Dog breeds dataset. It is not easy to train the alexnet model from 
scratch on the Dog breeds data itself. Curious minds can try for once to train Alexnet from scratch. We adopt Transfer Learning here. We 
obtain a pretrained Alexnet model trained on Imagenet and apply transfer learning to it to get better results.__

## Transfer Learning

In [ ]:
###########################################################################################
## TODO: Get pretrained AlexNet                                                          ##
## More details at https://pytorch.org/vision/stable/models.html                         ##
###########################################################################################



###########################################################################################
## TODO: Freeze the weigths of the pretrained alexnet model and change the last          ##
## classification layer from 1000 classes of Imagenet to 120 classes of Dog Breeds, only ##
## classification layer should be unfreezed and trainable                                ##
###########################################################################################




## Training the Network

You can re-use the training/validation loop source code provided in the first exercise notebook, or look at the `train_test.py` file in this exercise.



In [ ]:
## TODO: Create a loss function and optimizer


## TODO: Train the network (Hint: reuse provided code here)


## TODO: Evaluate results
## See: https://www.kaggle.com/competitions/dog-breed-identification/overview/evaluation

## Making Kaggle Submission

In [ ]:
##############################################################################################################
## Not So optimal Code: This can take upto 2 minutes to run: You are free to make an optimal version :)     ##
## It iterates over all test images to compute the softmax probablities from the last layer of the network  ##
##############################################################################################################
from transform import transform_testing
import PIL.Image
import torch.nn.functional as F
import numpy as np


augment_image = transform_testing()
test_data_root = dir_test + '/test_images/'
test_image_list = os.listdir(test_data_root) # list of test files 
result = []
for img_name in test_image_list:
    img = PIL.Image.open(test_data_root + img_name)
    img_tensor = augment_image(img)
    with torch.no_grad():
        output = net(img_tensor.unsqueeze_(0).cuda())
        probs = F.softmax(output, dim=1)
    result.append(probs.cpu().numpy())

all_predictions = np.concatenate(result)
print(all_predictions.shape)

In [ ]:
df = pd.DataFrame(all_predictions)
file_list = os.listdir(dir_converted_train) # list of classes to be provided here
df.columns = sorted(file_list)

# insert clean ids - without folder prefix and .jpg suffix - of images as first column
test_data_root = dir_test + 'test_images/' # list of all test files here
test_image_list = os.listdir(test_data_root)
df.insert(0, "id", [e[:-4] for e in test_image_list])
df.to_csv(f"sub_9_resnext.csv", index=False)

### Output ###

**TODO:** Please mention your final score on Kaggle and attach a screenshot of the Kaggle competetion result, showing your group name and your score.

## CHALLENGE  (optional)
Compete against each other, Come up with creative ideas. Try beating the score of __0.3__.


__Hints:__

1. Instead of Alexnet use pretrained resnet 18 model for better accuracy
2. Instead of a just adding the last classification layer, try adding two layers to get a better loss
3. Train some more layers at the end of the network with a very very small learning rate
4. Add Batch Normalizations or Dropout to the layers you have added, (If not present)
5. Add more augmentation to your dataset, see tranform.py file and use auto autoaugment to apply more rigorous data augmentation techniques